In [2]:
import deeplabcut

import yaml

from paramiko import SSHClient
import paramiko
from scp import SCPClient
import getpass

import os
import glob

In [2]:
# set shrek project folder and local folder (dest_folder)

shrek_project_folder = '/home/jts3256/DLC_videos/Han_202012-Joe-2021-01-10'
dest_folder = r'D:\Lab\Data\DLC_videos\Han_20201222_rwTwoPlanes'

In [3]:
# move folder from Shrek to local computer to label frames on local computer. Set shrek project folder and dest_folder
# Also may need to update ssh key/id
image_folder = shrek_project_folder + '/labeled-data'
config_file = shrek_project_folder + '/config.yaml'

if(not os.path.isdir(dest_folder)):
    os.mkdir(dest_folder)

ssh = SSHClient()
ssh.load_host_keys(filename = r'C:\Users\Joseph Sombeck\shrek_publickey (2).ppk')

pkey_filename = r'C:\Users\Joseph Sombeck\shrek_sshkey'
#pkey = paramiko.RSAKey.from_private_key_file(pkey_filename)
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())



ssh.connect(hostname = "FSMS270859X8810459.fsm.northwestern.edu", username= "jts3256", key_filename=pkey_filename)
with SCPClient(ssh.get_transport(), sanitize=lambda x: x) as scp:
    scp.get(image_folder,recursive = True, local_path=dest_folder)
    scp.get(config_file,recursive = False, local_path=dest_folder)
    
local_config_path = dest_folder+ r'\config.yaml'

# update project path in config path
with open(local_config_path) as file:
    data = yaml.load(file, Loader = yaml.FullLoader)

    data['project_path'] = dest_folder # default = 20

with open(local_config_path,'w') as file:
    yaml.dump(data,file)


In [ ]:
"""
label frames
"""
local_config_path= r'D:\Lab\Data\DLC_videos\Han_20201222_rwTwoPlanes\config.yaml'
    
#local_config_path = dest_folder + r'\config.yaml'

deeplabcut.label_frames(local_config_path,imtypes=['*png'])

In [6]:
data

{'Task': 'Crackle-20201203',
 'TrainingFraction': [0.95],
 'alphavalue': 0.7,
 'batch_size': 8,
 'bodyparts': ['shoulder',
  'elbow1',
  'elbow2',
  'wrist1',
  'wrist2',
  'hand1',
  'hand2',
  'hand3',
  'pointX',
  'pointY',
  'pointZ'],
 'colormap': 'plasma',
 'corner2move2': [50, 50],
 'croppedtraining': False,
 'cropping': False,
 'date': 'Jan25',
 'default_augmenter': 'default',
 'default_net_type': 'resnet_50',
 'dotsize': 12,
 'iteration': 0,
 'move2corner': True,
 'multianimalproject': False,
 'numframes2pick': 20,
 'pcutoff': 0.6,
 'project_path': 'D:\\Lab\\temp\\',
 'scorer': 'Joe',
 'skeleton': [['bodypart1', 'bodypart2'], ['objectA', 'bodypart3']],
 'skeleton_color': 'black',
 'snapshotindex': -1,
 'start': 0,
 'stop': 1,
 'video_sets': {'/home/jts3256/DLC_videos/Crackle-20201203-Joe-2021-01-25/videos/Crackle_20201203_00007.avi': {'crop': '0, 1280, 0, 1024'},
  '/home/jts3256/DLC_videos/Crackle-20201203-Joe-2021-01-25/videos/Crackle_20201203_00008.avi': {'crop': '0, 1280,

In [7]:
#deeplabcut.check_labels(local_config_path)

# move files back onto shrek
ssh = SSHClient()
ssh.load_host_keys(filename = r'C:\Users\Joseph Sombeck\shrek_publickey (2).ppk')

pkey_filename = r'C:\Users\Joseph Sombeck\shrek_sshkey'
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(hostname = "FSMS270859X8810459.fsm.northwestern.edu", username= "jts3256", key_filename=pkey_filename)
scp  = SCPClient(ssh.get_transport())

scp.put(files = dest_folder + '\labeled-data',recursive = True, 
        remote_path =shrek_project_folder)
